In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="MAP"
cutoff_to_optimize=10

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommenderClass = P3alphaRecommender



In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
study = optuna.create_study(study_name="P3_Hybrid_last", direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

[I 2024-01-09 21:42:49,870] A new study created in RDB with name: P3_Hybrid_last


In [15]:
def objective(trial):

    params = {
        'topK': trial.suggest_int('topK', 5, 1000),
        'alpha': trial.suggest_float('alpha', 0, 2),
    }

    MAP = 0
    for i in range(k_fold):
        recommender = recommenderClass(urm_train[i])
        recommender.fit(**params)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)

        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    return MAP

In [17]:
study.optimize(objective, n_trials=200)

P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 3598.52 column/sec. Elapsed time 6.18 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 6.12 sec. Users per second: 1642
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 3351.78 column/sec. Elapsed time 6.63 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 5.15 sec. Users per second: 1949
P3alphaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 3613.57 column/sec. Elapsed time 6.15 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 5.24 sec. Users per second: 1911
P3alphaRecommender: URM Detected 451 

[I 2024-01-10 00:21:07,141] Trial 122 finished with value: 0.025012179707851086 and parameters: {'topK': 98, 'alpha': 0.7782327038315002}. Best is trial 37 with value: 0.02618617452409684.


P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 1970.33 column/sec. Elapsed time 11.28 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 5.16 sec. Users per second: 1951
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 1901.60 column/sec. Elapsed time 11.69 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.87 sec. Users per second: 2062
P3alphaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 2748.65 column/sec. Elapsed time 8.08 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 5.61 sec. Users per second: 1785
P3alphaRecommender: URM Detected 45

[I 2024-01-10 00:25:00,774] Trial 126 finished with value: 0.025593367713541527 and parameters: {'topK': 370, 'alpha': 0.7874641137003433}. Best is trial 37 with value: 0.02618617452409684.


P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 2189.76 column/sec. Elapsed time 10.15 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 6.02 sec. Users per second: 1671
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 2606.73 column/sec. Elapsed time 8.52 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 6.73 sec. Users per second: 1491
P3alphaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 1961.53 column/sec. Elapsed time 11.33 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 6.14 sec. Users per second: 1632
P3alphaRecommender: URM Detected 45

...

[I 2024-01-10 07:02:14,190] Trial 651 finished with value: 0.02482848727615359 and parameters: {'topK': 82, 'alpha': 0.7749607611266207}. Best is trial 528 with value: 0.02666845662988516.


P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4272.63 column/sec. Elapsed time 5.20 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 5.55 sec. Users per second: 1813
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4361.82 column/sec. Elapsed time 5.09 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.68 sec. Users per second: 2144
P3alphaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 5043.58 column/sec. Elapsed time 4.41 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 4.17 sec. Users per second: 2403
P3alphaRecommender: URM Detected 451 

[I 2024-01-10 07:04:10,688] Trial 655 finished with value: 0.02629538477154662 and parameters: {'topK': 79, 'alpha': 0.6840774467012277}. Best is trial 528 with value: 0.02666845662988516.


P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4883.57 column/sec. Elapsed time 4.55 sec
EvaluatorHoldout: Processed 10057 (100.0%) in 4.46 sec. Users per second: 2254
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 7328.57 column/sec. Elapsed time 3.03 sec
EvaluatorHoldout: Processed 10040 (100.0%) in 4.22 sec. Users per second: 2381
P3alphaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 6928.33 column/sec. Elapsed time 3.21 sec
EvaluatorHoldout: Processed 10017 (100.0%) in 4.16 sec. Users per second: 2410
P3alphaRecommender: URM Detected 451 

[I 2024-01-10 07:05:41,466] Trial 658 finished with value: 0.024965024301158028 and parameters: {'topK': 21, 'alpha': 0.5538873501482211}. Best is trial 528 with value: 0.02666845662988516.


In [18]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4961.72 column/sec. Elapsed time 4.48 sec
EvaluatorHoldout: Processed 10492 (100.0%) in 3.73 sec. Users per second: 2815
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 6695.58 column/sec. Elapsed time 3.32 sec
EvaluatorHoldout: Processed 10461 (100.0%) in 5.25 sec. Users per second: 1991
P3alphaRecommender: URM Detected 479 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 345 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4672.70 column/sec. Elapsed time 4.76 sec
EvaluatorHoldout: Processed 10481 (100.0%) in 5.81 sec. Users per second: 1805
P3alphaRecommender: URM Detected 451 

0.03339475203160154

In [19]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = recommenderClass(urm_train_validation[i])
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

P3alphaRecommender: URM Detected 249 ( 2.0%) users with no interactions.
P3alphaRecommender: URM Detected 113 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4765.29 column/sec. Elapsed time 4.66 sec
EvaluatorHoldout: Processed 10492 (100.0%) in 4.29 sec. Users per second: 2443
P3alphaRecommender: URM Detected 191 ( 1.5%) users with no interactions.
P3alphaRecommender: URM Detected 101 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 5233.09 column/sec. Elapsed time 4.25 sec
EvaluatorHoldout: Processed 10461 (100.0%) in 5.02 sec. Users per second: 2084
P3alphaRecommender: URM Detected 235 ( 1.9%) users with no interactions.
P3alphaRecommender: URM Detected 105 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4601.78 column/sec. Elapsed time 4.83 sec
EvaluatorHoldout: Processed 10481 (100.0%) in 6.16 sec. Users per second: 1701
P3alphaRecommender: URM Detected 218 

0.04142907977977541